In [189]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy.stats
from scipy.optimize import minimize
from scipy.stats import multivariate_normal
from mpl_toolkits.mplot3d import Axes3D
import operator
from scipy.optimize import minimize
import math

In [190]:
def gen_dx(mu_real,sig_real,dt,N):
    N=np.int64(N)
    dW=np.sqrt(dt)*np.random.normal(0, 1, N)
    e= mu_real*dt+ sig_real*dW
    return(e)

In [191]:
#test generator
N=10000
dt= 0.01 
sig_real=10
mu_real=100
e=gen_dx(mu_real,sig_real,dt,N)

In [192]:
def liklihood(param,data, disc):
    L=0
    e=data
    N,dt=disc;
    N=np.int64(N)
    mu,sig =param;
    for i in range(0,N):
        L = L  - (e[i]-mu*dt )**2/(2*dt*sig**2) -0.5*np.log(2*math.pi*sig**2*dt)
    return(-1*L)

In [193]:
#test optimizer
bnds = ((None, None), (1e-5, None))
disc=np.array([1000,0.01])
args=(e,disc)
x0=np.array([13,7])
#min_param=minimize(liklihood,x0,args,method='SLSQP',bounds=bnds,\
                   #tol=1e-24,options={  'ftol': 1e-24,  'eps': 1e-12});min_param#
min_param=minimize(liklihood,x0,args,bounds=bnds);min_param

      fun: 1477.9733981593627
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00015916,  0.00200089])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 69
      nit: 19
   status: 0
  success: True
        x: array([97.11679522, 10.60814485])

In [194]:
def covergence_test(disc ,param_real, param_initial):
    
    N,dt=disc;
    mu_real,sig_real=param_real
    mu_initial,sig_initial=param_initial
    
    e=gen_dx(mu_real,sig_real,dt,N)
    
    args=(e,disc)
    x0=param_initial
    bnds = ((None, None), (1e-5, None))
    
    min_param=minimize(liklihood,x0,args,bounds=bnds);min_param
    #min_param=minimize(liklihood,x0,args,method='SLSQP',bounds=bnds,\
                   #tol=1e-24,options={  'ftol': 1e-24,  'eps': 1e-12});min_param
    error = np.divide ( abs(min_param.x - param_real),param_real )
    
    return(min_param.x, error)

In [195]:

disc_test=np.array([2000,10])
param_real_test=np.array([100,10])
param_initial_test= np.array([13,7])

covergence_test(disc_test,param_real_test,param_initial_test)


(array([100.1569011 ,   9.64297956]), array([0.00156901, 0.03570204]))

In [ ]:
#varying number of samples

N_test=np.array([10,50,100,500,1000,5000,10000,50000, 100000,500000, 1000000]) #np.arange(100,10000,1000)
err_fixeddt=np.zeros((len(N_test),2))
#res=np.zeros((len(N_test),2))

param_real_test=np.array([100,10])
param_initial_test= np.array([13,7])

fix_dt=10

for i in range(0,len(N_test)):
    disc_test=np.array([N_test[i],fix_dt])
    _,err_fixeddt[i,:]=covergence_test(disc_test,param_real_test,param_initial_test)


In [197]:
def abline(slope, intercept):
    """Plot a line from slope and intercept"""
    axes = plt.gca()
    x_vals = np.array(axes.get_xlim())
    y_vals = intercept + slope * x_vals
    plt.plot(x_vals, y_vals, '--', label='slope='+str(slope))

In [ ]:
plt.title('')
plt.xlabel('log(N)')
plt.ylabel('log(relative error)')
plt.plot(np.log10(N_test),  np.log10(err_fixeddt[:,0]), label='$\mu$')
plt.plot(np.log10(N_test),  np.log10(err_fixeddt[:,1]) , label='$\sigma$')
abline(-0.5,0)
plt.legend()
plt.savefig('case11test.pdf')

In [ ]:
#varying dt

dt_test=np.array([0.0000001,0.000001,0.00001,0.0001,0.001,0.005,0.01,0.05,0.1,0.5,1,10,100,1000])   #np.arange(0.001,2,0.2)

err_fixedN=np.zeros((len(dt_test),2))
#res1=np.zeros((len(dt_test),2))

param_real_test=np.array([100,10])
param_initial_test= np.array([13,7])

fix_N=10000

for i in range(0,len(dt_test)):
    disc_test=np.array([fix_N,dt_test[i]])
    _,err_fixedN[i,:]=covergence_test(disc_test,param_real_test,param_initial_test)

In [ ]:
plt.title('')
plt.xlabel('log(dt)')
plt.ylabel('log(relative error)')
plt.plot(np.log10(dt_test),  np.log10(err_fixedN[:,0]), label='$\mu$')
plt.plot(np.log10(dt_test),  np.log10(err_fixedN[:,1]), label='$\sigma$')
abline(-0.5,-3)
plt.legend()
plt.savefig('case12test.pdf')

In [ ]:
#varying number of samples - finite horizon

N_test=np.array([5,10,50,100,500,1000,5000,10000,50000,100000,500000, 1000000, 5000000])  #np.arange(100,10000,1000)
dt_test= np.divide(1,N_test)
err_var2=np.zeros((len(N_test),2))
#res=np.zeros((len(N_test),2))

param_real_test=np.array([100,10])
param_initial_test= np.array([13,7])

fix_dt=10

for i in range(0,len(N_test)):
    disc_test=np.array([N_test[i],dt_test[i] ] )
    _,err_var2[i,:]=covergence_test(disc_test,param_real_test,param_initial_test)

In [ ]:
plt.title('Finite Horizon')
plt.xlabel('$N$')
plt.ylabel('log(N)')
plt.plot(np.log10(N_test),  np.log10(err_var2[:,0]), label='$\mu$')
plt.plot(np.log10(N_test),  np.log10(err_var2[:,1]), label='$\sigma$')
abline(-0.5,0)
plt.legend()
plt.savefig('case13test344.pdf')

In [230]:
#varying number of samples infinte horizon

N_test=np.array([5,10,50,100,500,1000,5000,10000,50000,100000,500000, 1000000, 5000000])  #np.arange(100,10000,1000)
dt_test= np.divide(1,np.sqrt(N_test))
err_var1=np.zeros((len(N_test),2))
#res=np.zeros((len(N_test),2))

param_real_test=np.array([100,10])
param_initial_test= np.array([13,7])

fix_dt=10

for i in range(0,len(N_test)):
    disc_test=np.array([N_test[i],dt_test[i] ] )
    _,err_var1[i,:]=covergence_test(disc_test,param_real_test,param_initial_test)


KeyboardInterrupt: 

In [ ]:
plt.title('Infinite Horizon')
plt.xlabel('$log(N)$')
plt.ylabel('log(relative error)')
plt.plot(np.log10(N_test),  np.log10(err_var1[:,0]), label='$\mu$')
plt.plot(np.log10(N_test),  np.log10(err_var1[:,1]), label='$\sigma$')
abline(-0.5,0)
plt.legend()
plt.savefig('case13test3.pdf')